In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


### Importing libraries

In [2]:
from traitlets.traitlets import validate
import pandas as pd
import os
import json
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from matplotlib import pyplot as plt
import seaborn as sns
from pathlib import Path
import glob

In [3]:
#https://shubhasmitaroy.medium.com/for-those-who-get-an-error-on-importing-vggface-related-packages-e5afdd0c3f01
!pip install git+https://github.com/rcmalli/keras-vggface.git
!pip install keras_applications --no-deps
filename = "/usr/local/lib/python3.10/dist-packages/keras_vggface/models.py"
text = open(filename).read()
open(filename, "w+").write(text.replace('keras.engine.topology', 'tensorflow.keras.utils'))
import tensorflow as tf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-cb9q7rj6
  Running command git clone --filter=blob:none --quiet https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-cb9q7rj6
  Resolved https://github.com/rcmalli/keras-vggface.git to commit bee35376e76e35d00aeec503f2f242611a97b38a
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import os
import pandas as pd
import numpy as np
from keras_vggface.vggface import VGGFace
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split


In [5]:
path = "/content/gdrive/MyDrive/Colab Notebooks/Data/"

### Defining data paths

In [6]:
train_processed_dir = path+'Train/train_aligned/'
test_processed_dir = path+'Test/test_aligned/'

# remove rows that no face images
train_img = os.listdir(train_processed_dir)
test_img = os.listdir(test_processed_dir)

train = pd.read_csv(path+'Train/train.csv')
valid = pd.read_csv(path+'Test/valid.csv')

train = train.loc[train['name'].isin(train_img),:]
valid = valid.loc[valid['name'].isin(test_img),:]

print('train data dimension: {}'.format(str(train.shape)))
print('test data dimension:  {}'.format(str(valid.shape)))


train data dimension: (3203, 3)
test data dimension:  (750, 3)


In [ ]:
# create metrics, model dirs
Path(path+'/metrics').mkdir(parents = True, exist_ok = True)
Path(path+'/saved_model').mkdir(parents = True, exist_ok = True)

In [7]:
data = pd.concat([train, valid])
data[['gender','bmi','name']].head()

,gender,bmi,name
0,0,34.207396,img_0.bmp
1,0,26.453720,img_1.bmp
2,1,34.967561,img_2.bmp
3,1,22.044766,img_3.bmp
4,1,25.845588,img_6.bmp


In [8]:
!pip install mtcnn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Data Augmentation and Batch Generation 

In [10]:
# Create an ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(rescale=1./255)

# Generate batches of training images and labels
train_generator = datagen.flow_from_dataframe(
    dataframe=train,
    directory=train_processed_dir,
    x_col='name',
    y_col=['bmi', 'gender'],
    target_size=(224, 224),
    batch_size=32,
    class_mode='multi_output',
    shuffle=True
)


# Generate batches of validation images and labels
valid_generator = datagen.flow_from_dataframe(
    dataframe=valid,
    directory=test_processed_dir,
    x_col='name',
    y_col=['bmi', 'gender'],
    target_size=(224, 224),
    batch_size=32,
    class_mode='multi_output',
    shuffle=False
)


Found 3203 validated image filenames.
Found 750 validated image filenames.


### Model 1 : VGGFace model with ResNet50 and 64 fully connected layers

In [ ]:
# Load the pre-trained VGGFace model
base_model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3))

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add fully connected layers for BMI prediction
bmi_fc = Dense(64, activation='relu')(x)
bmi_fc = Dense(1, name='bmi')(bmi_fc)

# Add fully connected layers for gender prediction
gender_fc = Dense(64, activation='relu')(x)
gender_fc = Dense(1, activation='sigmoid', name='gender')(gender_fc)

# Create the fine-tuned model
model = Model(inputs=base_model.input, outputs=[bmi_fc, gender_fc])

# Freeze the weights of the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model with appropriate loss functions and metrics
model.compile(optimizer=Adam(), 
              loss={'bmi': 'mean_squared_error', 'gender': 'binary_crossentropy'},
              metrics={'bmi': 'mse', 'gender': 'accuracy'})

from keras.callbacks import EarlyStopping

# Train the model
model.fit(train_generator,
          steps_per_epoch=len(train_generator),
          epochs=10,
          validation_data=valid_generator,
          validation_steps=len(valid_generator),
          callbacks=[EarlyStopping(patience=2)])

model.save(path+'saved_model/fine_tuned_vggface_model_64.h5')


94694792/94694792 [==============================] - 1s 0us/step


##### Results

Epoch 1/10
101/101 [==============================] - 871s 9s/step - loss: 261.4586 - bmi_loss: 260.7733 - gender_loss: 0.6853 - bmi_mse: 260.7733 - gender_accuracy: 0.5832 - val_loss: 86.8630 - val_bmi_loss: 86.1779 - val_gender_loss: 0.6851 - val_bmi_mse: 86.1779 - val_gender_accuracy: 0.5680

Epoch 2/10
101/101 [==============================] - 714s 7s/step - loss: 64.9482 - bmi_loss: 64.2684 - gender_loss: 0.6797 - bmi_mse: 64.2684 - gender_accuracy: 0.5916 - val_loss: 86.1564 - val_bmi_loss: 85.4725 - val_gender_loss: 0.6839 - val_bmi_mse: 85.4725 - val_gender_accuracy: 0.5680

Epoch 3/10
101/101 [==============================] - 721s 7s/step - loss: 65.1290 - bmi_loss: 64.4527 - gender_loss: 0.6763 - bmi_mse: 64.4527 - gender_accuracy: 0.5935 - val_loss: 86.8603 - val_bmi_loss: 86.1715 - val_gender_loss: 0.6888 - val_bmi_mse: 86.1715 - val_gender_accuracy: 0.5680

Epoch 4/10
101/101 [==============================] - 717s 7s/step - loss: 65.1406 - bmi_loss: 64.4598 - gender_loss: 0.6808 - bmi_mse: 64.4598 - gender_accuracy: 0.5798 - val_loss: 88.2110 - val_bmi_loss: 87.5222 - val_gender_loss: 0.6888 - val_bmi_mse: 87.5222 - val_gender_accuracy: 0.5680

#### Validation

In [ ]:
from keras.models import load_model
from IPython.display import Image

image_name = 'img_3417.bmp'
image_path = path+'Test/Test_Img/'

# Load the saved model
model = load_model(path+'saved_model/fine_tuned_vggface_model_64.h5')

from keras_vggface.utils import preprocess_input
from PIL import Image

# Load and preprocess the input image
input_image = Image.open(image_path+image_name)
input_image = input_image.resize((224, 224))  # Resize to the input shape of the model
input_image = np.array(input_image).astype(np.float32)  # Convert to float32
input_image = preprocess_input(input_image)  # Preprocess the image

# Predict BMI and gender
prediction = model.predict(np.expand_dims(input_image, axis=0))

# Retrieve the predicted BMI and gender values
predicted_bmi = prediction[0][0]
predicted_gender = 'Male' if prediction[1][0] > 0.5 else 'Female'

# Print the predicted BMI and gender
print("Predicted BMI:", predicted_bmi)
print("Predicted Gender:", predicted_gender)

#22.49458302	Female	0	img_3417.bmp

1/1 [==============================] - 1s 1s/step
Predicted BMI: [50.503872]
Predicted Gender: Male


In [ ]:
print("Frozen layers with 64 fully connected dense layers")
model.evaluate(valid_generator)


Frozen layers with 64 fully connected dense layers
24/24 [==============================] - 135s 6s/step - loss: 88.2110 - bmi_loss: 87.5222 - gender_loss: 0.6888 - bmi_mse: 87.5222 - gender_accuracy: 0.5680


[88.21102142333984,
 87.52223205566406,
 0.6887925863265991,
 87.52223205566406,
 0.5680000185966492]

### Model 2 : VGGFace model with ResNet50 and 128 fully connected layers

In [ ]:
# Load the pre-trained VGGFace model
base_model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3))

# Add a global average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add fully connected layers for BMI prediction
bmi_fc = Dense(128, activation='relu')(x)
bmi_fc = Dense(1, name='bmi')(bmi_fc)

# Add fully connected layers for gender prediction
gender_fc = Dense(128, activation='relu')(x)
gender_fc = Dense(1, activation='sigmoid', name='gender')(gender_fc)

# Create the fine-tuned model
model = Model(inputs=base_model.input, outputs=[bmi_fc, gender_fc])

# Freeze the weights of the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model with appropriate loss functions and metrics
model.compile(optimizer=Adam(), 
              loss={'bmi': 'mean_squared_error', 'gender': 'binary_crossentropy'},
              metrics={'bmi': 'mse', 'gender': 'accuracy'})

# Train the model
model.fit(train_generator,
          steps_per_epoch=len(train_generator),
          epochs=20,
          validation_data=valid_generator,
          validation_steps=len(valid_generator),
          callbacks=[EarlyStopping(patience=5)])

# Save the fine-tuned model
model.save(path+'saved_model/fine_tuned_vggface_model_128.h5')


Epoch 1/20
101/101 [==============================] - 721s 7s/step - loss: 327.5560 - bmi_loss: 326.8747 - gender_loss: 0.6813 - bmi_mse: 326.8747 - gender_accuracy: 0.5791 - val_loss: 86.1752 - val_bmi_loss: 85.4357 - val_gender_loss: 0.7395 - val_bmi_mse: 85.4357 - val_gender_accuracy: 0.5680
Epoch 2/20
101/101 [==============================] - 785s 8s/step - loss: 64.9085 - bmi_loss: 64.2234 - gender_loss: 0.6851 - bmi_mse: 64.2234 - gender_accuracy: 0.5757 - val_loss: 87.4735 - val_bmi_loss: 86.7767 - val_gender_loss: 0.6967 - val_bmi_mse: 86.7767 - val_gender_accuracy: 0.5680
Epoch 3/20
101/101 [==============================] - 716s 7s/step - loss: 64.9831 - bmi_loss: 64.2989 - gender_loss: 0.6842 - bmi_mse: 64.2989 - gender_accuracy: 0.5851 - val_loss: 85.8994 - val_bmi_loss: 85.1724 - val_gender_loss: 0.7270 - val_bmi_mse: 85.1724 - val_gender_accuracy: 0.5680
Epoch 4/20
101/101 [==============================] - 712s 7s/step - loss: 65.0394 - bmi_loss: 64.3626 - gender_loss: 

#### Validation

In [ ]:
print("Frozen layers with 128 fully connected dense layers and 20 epochs with early stopping stop training if it doesn't improve for 5 consecutive epochs")
model.evaluate(valid_generator)

24/24 [==============================] - 133s 6s/step - loss: 93.5737 - bmi_loss: 92.8766 - gender_loss: 0.6971 - bmi_mse: 92.8766 - gender_accuracy: 0.5680


[93.57368469238281,
 92.87657165527344,
 0.6971324682235718,
 92.87657165527344,
 0.5680000185966492]

In [ ]:
from keras.models import load_model

# Load the saved model
model = load_model(path+'saved_model/fine_tuned_vggface_model_128.h5')

from keras_vggface.utils import preprocess_input
from PIL import Image

# Load and preprocess the input image
input_image = Image.open(image_path+image_name)
input_image = input_image.resize((224, 224))  # Resize to the input shape of the model
input_image = np.array(input_image).astype(np.float32)  # Convert to float32
input_image = preprocess_input(input_image)  # Preprocess the image

# Predict BMI and gender
prediction = model.predict(np.expand_dims(input_image, axis=0))

# Retrieve the predicted BMI and gender values
predicted_bmi = prediction[0][0]
predicted_gender = 'Male' if prediction[1][0] > 0.5 else 'Female'

# Print the predicted BMI and gender
print("Predicted BMI:", predicted_bmi)
print("Predicted Gender:", predicted_gender)

#22.49458302	Female	0	img_3417.bmp

1/1 [==============================] - 1s 1s/step
Predicted BMI: [43.087055]
Predicted Gender: Male


### Model 3 : VGGFace model with ResNet50 and 128 fully connected layers w unfreezing last 5 layers

In [ ]:
# Load the pre-trained VGGFace model
base_model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3))

# Unfreeze and train the last few layers of the base model
for layer in base_model.layers[-5:]:
    layer.trainable = True

# Add a global average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add fully connected layers for BMI prediction
bmi_fc = Dense(128, activation='relu')(x)
bmi_fc = Dense(1, name='bmi')(bmi_fc)

# Add fully connected layers for gender prediction
gender_fc = Dense(128, activation='relu')(x)
gender_fc = Dense(1, activation='sigmoid', name='gender')(gender_fc)

# Create the fine-tuned model
model = Model(inputs=base_model.input, outputs=[bmi_fc, gender_fc])

# Compile the model with appropriate loss functions and metrics
model.compile(optimizer=Adam(), 
              loss={'bmi': 'mean_squared_error', 'gender': 'binary_crossentropy'},
              metrics={'bmi': 'mse', 'gender': 'accuracy'})

# Train the model
model.fit(train_generator,
          steps_per_epoch=len(train_generator),
          epochs=20,
          validation_data=valid_generator,
          validation_steps=len(valid_generator),
          callbacks=[EarlyStopping(patience=5)])

# Save the fine-tuned model
model.save(path+'saved_model/fine_tuned_vggface_model_unfreeze5.h5')


Epoch 1/20
101/101 [==============================] - 2851s 28s/step - loss: 91.5627 - bmi_loss: 90.7616 - gender_loss: 0.8011 - bmi_mse: 90.7616 - gender_accuracy: 0.5882 - val_loss: 271465.2188 - val_bmi_loss: 271464.4062 - val_gender_loss: 0.8515 - val_bmi_mse: 271464.4062 - val_gender_accuracy: 0.4320
Epoch 2/20
101/101 [==============================] - 2814s 28s/step - loss: 65.2943 - bmi_loss: 64.6231 - gender_loss: 0.6712 - bmi_mse: 64.6231 - gender_accuracy: 0.6007 - val_loss: 98.3422 - val_bmi_loss: 97.6350 - val_gender_loss: 0.7072 - val_bmi_mse: 97.6350 - val_gender_accuracy: 0.5680
Epoch 3/20
101/101 [==============================] - 2844s 28s/step - loss: 57.9673 - bmi_loss: 57.2851 - gender_loss: 0.6821 - bmi_mse: 57.2851 - gender_accuracy: 0.6007 - val_loss: 504.5482 - val_bmi_loss: 503.8555 - val_gender_loss: 0.6927 - val_bmi_mse: 503.8555 - val_gender_accuracy: 0.5680
Epoch 4/20
101/101 [==============================] - 2838s 28s/step - loss: 56.3396 - bmi_loss: 55.

#### validation

In [ ]:
print("Unfreeze 5 layers with 128 fully connected dense layers and 20 epochs with early stopping stop training if it doesn't improve for 5 consecutive epochs")
model.evaluate(valid_generator)

Unfreeze 5 layers with 128 fully connected dense layers and 20 epochs with early stopping stop training if it doesn't improve for 5 consecutive epochs
24/24 [==============================] - 133s 5s/step - loss: 76.6980 - bmi_loss: 76.0428 - gender_loss: 0.6552 - bmi_mse: 76.0428 - gender_accuracy: 0.5680


[76.69803619384766,
 76.0428237915039,
 0.6552107334136963,
 76.0428237915039,
 0.5680000185966492]

### Model 4 : VGGFace model with VGG16 architecture and 64 fully connected layers 

In [ ]:
# Load the pre-trained VGGFace model
base_model = VGGFace(model='vgg16', include_top=False, input_shape=(224, 224, 3))

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add fully connected layers for BMI prediction
bmi_fc = Dense(64, activation='relu')(x)
bmi_fc = Dense(1, name='bmi')(bmi_fc)

# Add fully connected layers for gender prediction
gender_fc = Dense(64, activation='relu')(x)
gender_fc = Dense(1, activation='sigmoid', name='gender')(gender_fc)

# Create the fine-tuned model
model = Model(inputs=base_model.input, outputs=[bmi_fc, gender_fc])

# Freeze the weights of the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model with appropriate loss functions and metrics
model.compile(optimizer=Adam(), 
              loss={'bmi': 'mean_squared_error', 'gender': 'binary_crossentropy'},
              metrics={'bmi': 'mse', 'gender': 'accuracy'})

from keras.callbacks import EarlyStopping

# Train the model
model.fit(train_generator,
          steps_per_epoch=len(train_generator),
          epochs=10,
          validation_data=valid_generator,
          validation_steps=len(valid_generator),
          callbacks=[EarlyStopping(patience=5)])

model.save(path+'saved_model/fine_tuned_vggface_model_vgg16.h5')


58909280/58909280 [==============================] - 1s 0us/step
Epoch 1/10
101/101 [==============================] - 2602s 26s/step - loss: 915.5443 - bmi_loss: 915.0029 - gender_loss: 0.5414 - bmi_mse: 915.0029 - gender_accuracy: 0.7705 - val_loss: 733.5018 - val_bmi_loss: 733.1183 - val_gender_loss: 0.3834 - val_bmi_mse: 733.1183 - val_gender_accuracy: 0.9253
Epoch 2/10
101/101 [==============================] - 2578s 26s/step - loss: 378.4223 - bmi_loss: 378.1328 - gender_loss: 0.2896 - bmi_mse: 378.1328 - gender_accuracy: 0.9223 - val_loss: 233.6090 - val_bmi_loss: 233.3849 - val_gender_loss: 0.2241 - val_bmi_mse: 233.3849 - val_gender_accuracy: 0.9413
Epoch 3/10
101/101 [==============================] - 2563s 25s/step - loss: 129.3594 - bmi_loss: 129.1618 - gender_loss: 0.1976 - bmi_mse: 129.1618 - gender_accuracy: 0.9419 - val_loss: 153.0738 - val_bmi_loss: 152.8917 - val_gender_loss: 0.1821 - val_bmi_mse: 152.8917 - val_gender_accuracy: 0.9413
Epoch 4/10
101/101 [============

#### Validation

In [ ]:
model = load_model(path+'saved_model/fine_tuned_vggface_model_vgg16.h5')

print("VGG16 architecture w Frozen layers with 64 fully connected dense layers")
model.evaluate(valid_generator)


Unfreeze 5 layers with 128 fully connected dense layers and 20 epochs with early stopping stop training if it doesn't improve for 5 consecutive epochs
24/24 [==============================] - 484s 20s/step - loss: 116.8602 - bmi_loss: 116.7264 - gender_loss: 0.1338 - bmi_mse: 116.7264 - gender_accuracy: 0.9587


[116.8602294921875,
 116.7264175415039,
 0.1338060051202774,
 116.7264175415039,
 0.9586666822433472]

In [ ]:
from keras_vggface import utils
from keras.utils import load_img, img_to_array
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np

def crop_img(im,x,y,w,h):
    return im[y:(y+h),x:(x+w),:]

def detect_face(face_path):
    img = cv2.cvtColor(cv2.imread(face_path), cv2.COLOR_BGR2RGB)
    box = detector.detect_faces(img)
    return box

def cut_negative_boundary(box):
    res = []
    for x in box['box']:
        if x < 0:
            x = 0
        res.append(x)
    box['box'] = res
    return box


In [ ]:
import cv2
import numpy as np

def preprocess_image(image_path):
    # Load the image using OpenCV
    image = cv2.imread(image_path)    
    
    # Resize the cropped face image to a consistent size (e.g., 224x224 pixels)
    resized_image = cv2.resize(image, (224, 224))
    
    # Convert the image to grayscale or RGB based on the model's input requirements
    rgb_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
    
    # Normalize the pixel values to be in the range [0, 1]
    normalized_image = rgb_image / 255.0
    
    # Prepare the input data as a numpy array or tensor
    preprocessed_image = np.expand_dims(normalized_image, axis=0)
    
    return preprocessed_image


In [ ]:
train.head()
train_processed_dir = path+'Train/train_aligned/'
test_processed_dir = path+'Test/test_aligned/'

In [ ]:
import pandas as pd
import os
import numpy as np


# Prepare train data
train_images = []
train_bmi_labels = []
train_gender_labels = []

for index, row in train.iterrows():
    #image_path = os.path.join(path+'Train/train_aligned/', row['name'])
    #preprocessed_image = preprocess_image(image_path)
    
    #train_images.append(preprocessed_image)
    train_bmi_labels.append(row['bmi'])
    train_gender_labels.append(row['gender'])

#train_images = np.array(train_images)
train_bmi_labels = np.array(train_bmi_labels)
train_gender_labels = np.array(train_gender_labels)

# Prepare test data
test_images = []
test_bmi_labels = []
test_gender_labels = []

for index, row in valid.iterrows():
    #image_path = os.path.join(path+'Test/test_aligned/', row['name'])
    #preprocessed_image = preprocess_image(image_path)
    
    #test_images.append(preprocessed_image)
    test_bmi_labels.append(row['bmi'])
    test_gender_labels.append(row['gender'])

#test_images = np.array(test_images)
test_bmi_labels = np.array(test_bmi_labels)
test_gender_labels = np.array(test_gender_labels)


In [ ]:
test_images.shape

(750, 224, 224, 3)

In [ ]:
train_bmi_labels.shape

(3203,)

In [ ]:
train_images = np.squeeze(train_images, axis=1)
test_images = np.squeeze(test_images, axis=1)

In [ ]:
# Save the train_images array to a file
np.save(path+'train_images.npy', train_images)
np.save(path+'test_images.npy', test_images)

In [ ]:
# Load the saved train_images array
train_images = np.load(path+'train_images.npy')
test_images = np.load(path+'test_images.npy')


### Model 5 : CNN model to predict BMI and Gender

The purpose of training the model using gender information as an additional input and predicting gender again as one of the outputs is to create a multi-task learning setup. In this scenario, the model is trained not only to predict BMI but also to predict gender simultaneously.

In [ ]:
from keras.layers import Concatenate

input_image = tf.keras.layers.Input(shape=(224, 224, 3), name='input_image')
input_gender = tf.keras.layers.Input(shape=(1,), name='input_gender')

conv1 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu')(input_image)
pool1 = tf.keras.layers.MaxPooling2D(pool_size=2)(conv1)

conv2 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu')(pool1)
pool2 = tf.keras.layers.MaxPooling2D(pool_size=2)(conv2)

conv3 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu')(pool2)
pool3 = tf.keras.layers.MaxPooling2D(pool_size=2)(conv3)

flatten = tf.keras.layers.Flatten()(pool3)

# Gender branch
dense_gender = tf.keras.layers.Dense(units=32, activation='relu')(input_gender)

# Concatenate the image and gender features
concatenated = tf.keras.layers.Concatenate()([flatten, dense_gender])

# Output layers
dense1 = tf.keras.layers.Dense(units=32, activation='relu')(concatenated)
output_BMI = tf.keras.layers.Dense(units=1, activation='relu', name='output_BMI')(dense1)
output_gender = tf.keras.layers.Dense(units=1, activation='sigmoid', name='output_gender')(dense1)

# Create the model
model_BMI_gender_cnn = tf.keras.Model(inputs=[input_image, input_gender], outputs=[output_BMI, output_gender])

# Compile the model
model_BMI_gender_cnn.compile(loss=['mse', 'binary_crossentropy'], optimizer='adam', metrics=['mae', 'accuracy'])

# Fit the model to the training data
model_BMI_gender_cnn.fit([train_images, train_gender_labels], [np.log(train_bmi_labels), train_gender_labels], epochs=13, batch_size=1)


Epoch 1/13
3203/3203 [==============================] - 487s 151ms/step - loss: 12.0406 - output_BMI_loss: 11.9630 - output_gender_loss: 0.0776 - output_BMI_mae: 3.4508 - output_BMI_accuracy: 0.0000e+00 - output_gender_mae: 0.0530 - output_gender_accuracy: 0.9819

Epoch 2/13
3203/3203 [==============================] - 487s 152ms/step - loss: 11.9631 - output_BMI_loss: 11.9629 - output_gender_loss: 1.4063e-04 - output_BMI_mae: 3.4508 - output_BMI_accuracy: 0.0000e+00 - output_gender_mae: 1.4061e-04 - output_gender_accuracy: 1.0000

Epoch 3/13
3203/3203 [==============================] - 486s 152ms/step - loss: 11.9630 - output_BMI_loss: 11.9630 - output_gender_loss: 1.6620e-05 - output_BMI_mae: 3.4508 - output_BMI_accuracy: 0.0000e+00 - output_gender_mae: 1.6619e-05 - output_gender_accuracy: 1.0000

Epoch 4/13
3203/3203 [==============================] - 483s 151ms/step - loss: 11.9630 - output_BMI_loss: 11.9630 - output_gender_loss: 2.5776e-06 - output_BMI_mae: 3.4508 - output_BMI_accuracy: 0.0000e+00 - output_gender_mae: 2.5771e-06 - output_gender_accuracy: 1.0000

Epoch 5/13
3203/3203 [==============================] - 483s 151ms/step - loss: 11.9630 - output_BMI_loss: 11.9630 - output_gender_loss: 4.2830e-07 - output_BMI_mae: 3.4508 - output_BMI_accuracy: 0.0000e+00 - output_gender_mae: 4.2818e-07 - output_gender_accuracy: 1.0000

Epoch 6/13
3203/3203 [==============================] - 482s 151ms/step - loss: 11.9630 - output_BMI_loss: 11.9630 - output_gender_loss: 7.3262e-08 - output_BMI_mae: 3.4508 - output_BMI_accuracy: 0.0000e+00 - output_gender_mae: 7.1090e-08 - output_gender_accuracy: 1.0000

Epoch 7/13
3203/3203 [==============================] - 483s 151ms/step - loss: 11.9629 - output_BMI_loss: 11.9629 - output_gender_loss: 1.5085e-08 - output_BMI_mae: 3.4508 - output_BMI_accuracy: 0.0000e+00 - output_gender_mae: 1.1093e-08 - output_gender_accuracy: 1.0000

Epoch 8/13
3203/3203 [==============================] - 482s 151ms/step - loss: 11.9630 - output_BMI_loss: 11.9630 - output_gender_loss: 5.6973e-09 - output_BMI_mae: 3.4508 - output_BMI_accuracy: 0.0000e+00 - output_gender_mae: 1.9156e-09 - output_gender_accuracy: 1.0000

Epoch 9/13
3203/3203 [==============================] - 482s 150ms/step - loss: 11.9630 - output_BMI_loss: 11.9630 - output_gender_loss: 3.2846e-09 - output_BMI_mae: 3.4508 - output_BMI_accuracy: 0.0000e+00 - output_gender_mae: 4.7337e-10 - output_gender_accuracy: 1.0000

Epoch 10/13
3203/3203 [==============================] - 482s 150ms/step - loss: 11.9629 - output_BMI_loss: 11.9629 - output_gender_loss: 3.0057e-09 - output_BMI_mae: 3.4508 - output_BMI_accuracy: 0.0000e+00 - output_gender_mae: 1.6060e-10 - output_gender_accuracy: 1.0000

Epoch 11/13
3203/3203 [==============================] - 482s 150ms/step - loss: 11.9629 - output_BMI_loss: 11.9629 - output_gender_loss: 2.3996e-09 - output_BMI_mae: 3.4508 - output_BMI_accuracy: 0.0000e+00 - output_gender_mae: 7.5919e-11 - output_gender_accuracy: 1.0000

Epoch 12/13
3203/3203 [==============================] - 481s 150ms/step - loss: 11.9629 - output_BMI_loss: 11.9629 - output_gender_loss: 3.1511e-09 - output_BMI_mae: 3.4508 - output_BMI_accuracy: 0.0000e+00 - output_gender_mae: 3.8642e-11 - output_gender_accuracy: 1.0000

Epoch 13/13
3203/3203 [==============================] - 481s 150ms/step - loss: 11.9629 - output_BMI_loss: 11.9629 - output_gender_loss: 2.4407e-09 - output_BMI_mae: 3.4508 - output_BMI_accuracy: 0.0000e+00 - output_gender_mae: 1.9102e-11 - output_gender_accuracy: 1.0000

24/24 [==============================] - 30s 1s/step - loss: 12.1983 - output_BMI_loss: 12.1983 - output_gender_loss: 5.0526e-09 - output_BMI_mae: 3.4833 - output_BMI_accuracy: 0.0000e+00 - output_gender_mae: 1.9673e-12 - output_gender_accuracy: 1.0000


#### Validation

In [ ]:
# Evaluate the model on the test data
predictions = model_BMI_gender_cnn.evaluate([test_images, test_gender_labels], [np.log(test_bmi_labels), test_gender_labels])

model_BMI_gender_cnn.save(path+'saved_model/cnn_BMI_gender.h5')


24/24 [==============================] - 30s 1s/step - loss: 12.1983 - output_BMI_loss: 12.1983 - output_gender_loss: 5.0526e-09 - output_BMI_mae: 3.4833 - output_BMI_accuracy: 0.0000e+00 - output_gender_mae: 1.9673e-12 - output_gender_accuracy: 1.0000


In [ ]:
test_loss = predictions[0]
gender_loss = predictions[2]
test_BMI_mae = predictions[3]
test_gender_accuracy = predictions[-1]

# Print the results
print('Loss (BMI):', test_loss)
print('MAE (BMI):', test_BMI_mae)
print('Loss (Gender):', gender_loss)
print('Accuracy (Gender):', test_gender_accuracy)

Loss (BMI): 12.198328018188477
MAE (BMI): 3.483299493789673
Loss (Gender): 5.052609441946743e-09
Accuracy (Gender): 1.0
